In [1]:
import dateparser
import datetime
from collections import namedtuple
import CTLReader
import OBSReader
import os
import re
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
def extract_obs_data(path,filelist):
    records = []
    for file in filelist:
        obs = OBSReader.obs_from_file(path+os.sep+file)
        otime = OBSReader.P.search(file).group(2)
        time = dateparser.parse("20%s/%s/%s-%s:00:00"%(otime[:2],otime[2:4],otime[4:6],otime[6:8]))
        if hasattr(obs,'mean'):
            records.append(ObsSummary(
                date=time,
                temperature=obs.mean['temperatura'],
                dewp_temperature=obs.mean['dewp_temperatura'],
                pressure=obs.mean['pressure'],
                psl=obs.mean['psl'],
                humidity=obs.mean['humidity'],
                u_wind=obs.mean['u_wind'],
                v_wind=obs.mean['v_wind']))
            print('one record')
    return records

In [3]:
def derive_ntime_feature(df,feature,N):
    rows = df.shape[0]
    ntime_prior_measurements = [None]*N + [df[feature][i-N] for i in range(N,rows)]
    col_name = "{}_{}".format(feature,N)
    df[col_name] = ntime_prior_measurements

In [4]:
def forcast_in_guangdong(ctl,file):
    fcst = CTLReader.CTLReader(ctl,file)
    lon,lat=np.meshgrid(fcst.variables['longitude'][:].flatten(),
                        fcst.variables['latitude'][:].flatten())
    res = {}
    res['latitude'] = lat.flatten()
    res['longitude']= lon.flatten()
    res=pd.DataFrame(res)
    geometry = [Point(xy) for xy in zip(res.longitude,res.latitude)]
    crs = {'init':'epsg:4326'}
    data = gpd.GeoDataFrame(res,crs=crs,geometry=geometry)
    OBSReader.GUANGDONG.crs = data.crs
    datainner=gpd.sjoin(data,OBSReader.GUANGDONG,how='inner',op='within')
    return datainner.drop(['index_right','2000','DZZSTLSMJ'
                          ,'QLDJ','QLXS','XZQMC'],axis=1)

In [5]:
def forcast_to_result(ctl,file,lonlat_info):
    fcst = CTLReader.CTLReader(ctl,file)
    res = {}
    for var in need_to_read:
        res[var]=fcst.variables[var][:].flatten()
    res=pd.concat((lonlat_info,pd.DataFrame(res)),axis=1).dropna()
    return res.mean()

In [6]:
#获取观测结果
features   = ['date','temperature','dewp_temperature',
              'pressure','psl','humidity','u_wind','v_wind']
ObsSummary = namedtuple("ObsSummary",features)
need_to_read = ['t2m','ps','psl','u10m','v10m','rh2m','ts','cr']
ctl = os.path.abspath('./../read_binary/test.ctl')
file = os.path.abspath('./../read_binary/mars3km17040100001')
fcstpath = os.path.abspath('D:\data_mars3km_00f72/')
lonlat_in_guangdong = forcast_in_guangdong(ctl,file)

In [7]:
path = '/home/wenqs/data_synop04'
filelist = os.listdir(path)
filelist.sort()

FileNotFoundError: [Errno 2] No such file or directory: '/home/wenqs/data_synop04'

In [6]:
records = extract_obs_data(path,filelist)

/home/wenqs/project_mlwf/main/OBSReader.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col][missing_vals] = df[col+'_b'][missing_vals]
/home/wenqs/project_mlwf/main/OBSReader.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col][missing_vals] = df[col+'_a'][missing_vals]
/home/wenqs/project_mlwf/main/OBSReader.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col][missing_vals] = df[col+'_b'][missing_vals]
/home/wenqs/project_mlwf/main/OBSReader.py:65: SettingWith

In [7]:
import pickle
#with open('records.pkl','wb') as fp:
#    pickle.dump(records,fp)

In [8]:
with open('records.pkl','rb') as fp:
    records=pickle.load(fp)

In [9]:
df = pd.DataFrame(records,columns=features).set_index('date')

In [10]:
fcst_features   = ['date','CombinedRadarReflective','SurfacePressure',
                   'SeaLevelPressure','RelativeHumidity2m','Temperature2m',
                   'SurfaceTemperature','U_wind10m','V_wind10m',
                   'CombinedRadarReflective_prefcst','SurfacePressure_prefcst',
                   'SeaLevelPressure_prefcst','RelativeHumidity2m_prefcst','Temperature2m_prefcst',
                   'SurfaceTemperature_prefcst','U_wind10m_prefcst','V_wind10m_prefcst']
FcstSummary = namedtuple("FcstSummary",fcst_features)

In [11]:
forcast_d = datetime.timedelta(days=1)
fcstpath = os.path.abspath('D:\data_mars3km_00f72/')

In [249]:
fcst_records=[]
for date in df.index:
    validtime = date.to_pydatetime()
    fcst_date_before=date.date() - forcast_d
    fcst_date=dateparser.parse(date.date().strftime("%Y/%m/%d-%H:%M:%S"))
    fcst_date_before=dateparser.parse(fcst_date_before.strftime("%Y/%m/%d-%H:%M:%S"))
    
    timedelta = validtime - fcst_date
    timedelta_before = validtime - fcst_date_before
    
    timedelta=int(timedelta.total_seconds()/3600)
    timedelta_before=int(timedelta_before.total_seconds()/3600)
    
    filename1=fcstpath+"%s%03d"%(fcst_date.strftime("%Y%m%d%H/mars3km%y%m%d%H"),timedelta)
    filename2=fcstpath+"%s%03d"%(fcst_date_before.strftime("%Y%m%d%H/mars3km%y%m%d%H"),timedelta_before)
    if os.path.isfile(filename1) and os.path.isfile(filename2):
        f1=forcast_to_result(ctl,filename1,lonlat_in_guangdong)
        f2=forcast_to_result(ctl,filename2,lonlat_in_guangdong)
        fcst_records.append(FcstSummary(
            date                   =validtime,
            CombinedRadarReflective=f1.cr,
            SurfacePressure        =f1.ps,
            SeaLevelPressure       =f1.psl,
            RelativeHumidity2m     =f1.rh2m,
            Temperature2m          =f1.t2m,
            SurfaceTemperature     =f1.ts,
            U_wind10m              =f1.u10m,
            V_wind10m              =f1.v10m,
            CombinedRadarReflective_prefcst=f2.cr,
            SurfacePressure_prefcst        =f2.ps,
            SeaLevelPressure_prefcst       =f2.psl,
            RelativeHumidity2m_prefcst     =f2.rh2m,
            Temperature2m_prefcst          =f2.t2m,
            SurfaceTemperature_prefcst     =f2.ts,
            U_wind10m_prefcst              =f2.u10m,
            V_wind10m_prefcst              =f2.v10m,
        ))

time: 1
time: 2
time: 3
time: 4
time: 5
time: 6
time: 7
time: 8
time: 9
time: 10
time: 11
time: 12
time: 13
time: 14
time: 15
time: 16
time: 17
time: 18
time: 19
time: 20
time: 21
time: 22
time: 23
time: 24
time: 25
time: 26
time: 27
time: 28
time: 29
time: 30
time: 31
time: 32
time: 33
time: 34
time: 35
time: 36
time: 37
time: 38
time: 39
time: 40
time: 41
time: 42
time: 43
time: 44
time: 45
time: 46
time: 47
time: 48
time: 49
time: 50
time: 51
time: 52
time: 53
time: 54
time: 55
time: 56
time: 57
time: 58
time: 59
time: 60
time: 61
time: 62
time: 63
time: 64
time: 65
time: 66
time: 67
time: 68
time: 69
time: 70
time: 71
time: 72
time: 73
time: 74
time: 75
time: 76
time: 77
time: 78
time: 79
time: 80
time: 81
time: 82
time: 83
time: 84
time: 85
time: 86
time: 87
time: 88
time: 89
time: 90
time: 91
time: 92
time: 93
time: 94
time: 95
time: 96
time: 97
time: 98
time: 99
time: 100
time: 101
time: 102
time: 103
time: 104
time: 105
time: 106
time: 107
time: 108
time: 109
time: 110
time: 11

In [252]:
#with open('fcst_records.pkl','wb') as fp:
#    pickle.dump(fcst_records,fp)

In [12]:
with open('fcst_records.pkl','rb') as fp:
    fcst_records=pickle.load(fp)

In [13]:
fcst_df = pd.DataFrame(fcst_records,columns=fcst_features).set_index('date')

#暂时不考虑雷达反射率回波
fcst_df = fcst_df.drop(['CombinedRadarReflective','CombinedRadarReflective_prefcst'],axis=1)

In [14]:
#分别获取观测与预报的分类信息
features_obs = list(df)
features_fcst= list(fcst_df)

In [15]:
#合并两份数据集
df_all = pd.concat([df,fcst_df],axis=1).dropna()
# 如果完全不引入模式预报
#df = df.dropna()
features_all=list(df_all)

In [16]:
#获取纯的模式预报结果
only_fcst=fcst_df[['Temperature2m','Temperature2m_prefcst']]

In [17]:
for feature in features_all:
    if feature != 'date':
        for N in range(1,4):
            derive_ntime_feature(df_all,feature,N)

In [18]:
# 排除不需要的实时变量  features_all
# 或者可以考虑只排除预报时刻的已知观测信息 features_obs
to_remove = [feature for feature in features_obs if feature not in ['temperature']]

In [19]:
to_save = [col for col in df_all.columns if col not in to_remove]
df_all = df_all[to_save]

In [20]:
spread = df_all.describe().T
IQR = spread['75%'] - spread['25%']
spread['outliers'] = (spread['min']<(spread['25%']-(3*IQR)))|(spread['max']>(spread['75%']+3*IQR))
spread.loc[spread.outliers,]

,count,mean,std,min,25%,50%,75%,max,outliers
RelativeHumidity2m,671.0,80.209165,21.220256,0.0000,75.295364,85.513954,94.213116,99.144684,True
u_wind_1,670.0,-0.421079,0.536775,-2.3168,-0.700493,-0.439780,-0.175425,1.426029,True
u_wind_2,669.0,-0.421076,0.537176,-2.3168,-0.700500,-0.440000,-0.174928,1.426029,True
u_wind_3,668.0,-0.420883,0.537555,-2.3168,-0.700540,-0.439780,-0.174462,1.426029,True
RelativeHumidity2m_1,670.0,80.190278,21.230465,0.0000,75.292080,85.508579,94.217184,99.144684,True
RelativeHumidity2m_2,669.0,80.169330,21.239419,0.0000,75.288795,85.503204,94.221252,99.144684,True
RelativeHumidity2m_3,668.0,80.149036,21.248843,0.0000,75.271202,85.473812,94.233212,99.144684,True


In [21]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [24]:
df_all['RelativeHumidity2m'].hist(label='RelativeHumidity2m')
df_all['RelativeHumidity2m_prefcst'].hist(label='RelativeHumidity2m_prefcst')
plt.title('distribution of RelativeHumidity2m(0-24) and RelativeHumidity2m_prefcst(24-48)')

Text(0.5,1,'distribution of RelativeHumidity2m(0-24) and RelativeHumidity2m_prefcst(24-48)')

In [22]:
df_all = df_all.dropna()

In [23]:
corr=df_all.corr()[['temperature']].sort_values('temperature')
corr=corr[abs(corr)>0.6]
corr=corr.dropna()

In [24]:
print(corr)

                              temperature
SeaLevelPressure                -0.671090
SeaLevelPressure_prefcst        -0.656539
SurfacePressure                 -0.645180
SeaLevelPressure_1              -0.630928
SurfacePressure_prefcst         -0.629470
SeaLevelPressure_prefcst_1      -0.613689
SurfacePressure_1               -0.602984
U_wind10m                        0.605368
U_wind10m_3                      0.607103
U_wind10m_2                      0.612556
U_wind10m_1                      0.613194
v_wind_1                         0.616498
U_wind10m_prefcst_3              0.643260
U_wind10m_prefcst_2              0.657663
U_wind10m_prefcst_1              0.669611
U_wind10m_prefcst                0.676848
Temperature2m_prefcst_3          0.792047
SurfaceTemperature_prefcst_3     0.807475
temperature_3                    0.817180
Temperature2m_3                  0.820444
SurfaceTemperature_3             0.839467
Temperature2m_prefcst_2          0.871124
SurfaceTemperature_prefcst_2     0

In [25]:
predictors = list(corr.drop(['temperature','v_wind_1','temperature_3','temperature_2','temperature_1'],axis=0).T)

In [26]:
predictors

['SeaLevelPressure',
 'SeaLevelPressure_prefcst',
 'SurfacePressure',
 'SeaLevelPressure_1',
 'SurfacePressure_prefcst',
 'SeaLevelPressure_prefcst_1',
 'SurfacePressure_1',
 'U_wind10m',
 'U_wind10m_3',
 'U_wind10m_2',
 'U_wind10m_1',
 'U_wind10m_prefcst_3',
 'U_wind10m_prefcst_2',
 'U_wind10m_prefcst_1',
 'U_wind10m_prefcst',
 'Temperature2m_prefcst_3',
 'SurfaceTemperature_prefcst_3',
 'Temperature2m_3',
 'SurfaceTemperature_3',
 'Temperature2m_prefcst_2',
 'SurfaceTemperature_prefcst_2',
 'SurfaceTemperature_prefcst',
 'SurfaceTemperature',
 'Temperature2m_2',
 'SurfaceTemperature_prefcst_1',
 'SurfaceTemperature_2',
 'Temperature2m_prefcst_1',
 'Temperature2m_prefcst',
 'SurfaceTemperature_1',
 'Temperature2m_1',
 'Temperature2m']

In [ ]:
#只考虑绝对值在0.6以上相关的变量，并且舍弃前一个观测时次的温度值

In [27]:
#predictors = ['temperatura_2','temperatura_3',
#              'dewp_temperatura_1','dewp_temperatura_2','dewp_temperatura_3',
#             'v_wind_1','psl_1']

df2 = df_all[['temperature'] + predictors]
df2 = df2.dropna()

In [28]:
import statsmodels.api as sm

X = df2[predictors]
y = df2['temperature']
X = sm.add_constant(X)
X.iloc[:5,:5]

C:\Users\wenqi\Anaconda3\envs\geopandas\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


,const,SeaLevelPressure,SeaLevelPressure_prefcst,SurfacePressure,SeaLevelPressure_1
date,,,,,
2017-04-01 03:00:00,1.0,1021.478699,1020.904419,1000.673523,1021.467590
2017-04-01 04:00:00,1.0,1021.223572,1020.327332,1000.481262,1021.478699
2017-04-01 05:00:00,1.0,1020.641357,1019.782043,999.947266,1021.223572
2017-04-01 06:00:00,1.0,1020.022400,1019.292725,999.358521,1020.641357
2017-04-01 07:00:00,1.0,1019.498962,1018.864990,998.842834,1020.022400


In [29]:
vf = True
while vf is True:
    model = sm.OLS(y,X).fit()
    pmax  = model.pvalues.max()
    if pmax > 0.05:
        X=X.drop(model.pvalues.idxmax(),axis=1)
    else:
        vf = False

In [30]:
X

,const,SeaLevelPressure_prefcst,SurfacePressure_prefcst,U_wind10m_1,U_wind10m_prefcst_3,U_wind10m_prefcst_2,U_wind10m_prefcst,Temperature2m_prefcst_3,SurfaceTemperature_prefcst_3,Temperature2m_3,Temperature2m_prefcst_2,SurfaceTemperature_prefcst_2,SurfaceTemperature_prefcst,Temperature2m
date,,,,,,,,,,,,,,
2017-04-01 03:00:00,1.0,1020.904419,1000.101624,-3.361407,-3.351451,-3.554580,-3.502501,286.525116,287.688507,287.477295,288.177856,289.928650,293.958038,291.890320
2017-04-01 04:00:00,1.0,1020.327332,999.595276,-3.659685,-3.554580,-3.543207,-3.476880,288.177856,289.928650,288.587433,289.854004,292.076172,295.417419,293.275085
2017-04-01 05:00:00,1.0,1019.782043,999.099426,-3.798199,-3.543207,-3.502501,-3.479424,289.854004,292.076172,290.279663,291.488586,293.958038,296.365479,294.355225
2017-04-01 06:00:00,1.0,1019.292725,998.638855,-3.903121,-3.502501,-3.476880,-3.495173,291.488586,293.958038,291.890320,292.948853,295.417419,296.815887,295.102448
2017-04-01 07:00:00,1.0,1018.864990,998.218994,-4.022787,-3.476880,-3.479424,-3.503056,292.948853,295.417419,293.275085,294.109497,296.365479,296.774719,295.485321
2017-04-01 08:00:00,1.0,1018.815308,998.149292,-4.130190,-3.479424,-3.495173,-3.570184,294.109497,296.365479,294.355225,294.923157,296.815887,296.210968,295.482849
2017-04-01 09:00:00,1.0,1018.774048,998.065918,-4.206345,-3.495173,-3.503056,-3.609655,294.923157,296.815887,295.102448,295.365021,296.774719,295.124908,295.022797
2017-04-01 10:00:00,1.0,1018.910339,998.133728,-4.171447,-3.503056,-3.570184,-3.204129,295.365021,296.774719,295.485321,295.411652,296.210968,293.549713,293.950836
2017-04-01 11:00:00,1.0,1019.248962,998.359131,-3.583307,-3.570184,-3.609655,-2.863504,295.411652,296.210968,295.482849,294.997192,295.124908,291.053925,291.706238


In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X = X.drop('const',axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=12)


In [33]:
from sklearn.linear_model import LinearRegression

In [34]:
regressor = LinearRegression()
regressor.fit(X_train,y_train)
prediction = regressor.predict(X_test)

In [37]:
with open('regressor.pkl','wb') as fp:
    pickle.dump(regressor,fp)

with open('X_test.pkl','wb') as fp:
    pickle.dump(X_test,fp)

In [98]:
res = pd.concat([y_test,pd.DataFrame({'end var':prediction},index=y_test.index)],axis=1)
res = pd.concat([res,only_fcst],axis=1).dropna()
res = res - 273.15

In [99]:
res.plot()

In [88]:
pd.concat([res,only_fcst],axis=1)

,temperature,end var,Temperature2m,Temperature2m_prefcst
date,,,,
2017-04-01 00:00:00,NaN,NaN,287.477295,286.525116
2017-04-01 01:00:00,NaN,NaN,288.587433,288.177856
2017-04-01 02:00:00,NaN,NaN,290.279663,289.854004
2017-04-01 03:00:00,NaN,NaN,291.890320,291.488586
2017-04-01 04:00:00,NaN,NaN,293.275085,292.948853
2017-04-01 05:00:00,NaN,NaN,294.355225,294.109497
2017-04-01 06:00:00,295.432222,294.751549,295.102448,294.923157
2017-04-01 07:00:00,NaN,NaN,295.485321,295.365021
2017-04-01 08:00:00,NaN,NaN,295.482849,295.411652


In [99]:
fin[['temperature','Temperature2m','Temperature2m_prefcst']].sort_index().plot()

In [152]:
fin[['temperature','end var']].sort_index().plot()

In [94]:
from sklearn.metrics import mean_absolute_error,median_absolute_error

In [101]:
print('线性方案的平均绝对误差：',mean_absolute_error(res['temperature'],res['end var']),
      '线性方案的绝对误差中位数：',median_absolute_error(res['temperature'],res['end var']))
print('纯3km数值模式的24小时平均绝对误差：',mean_absolute_error(res['temperature'],res['Temperature2m']),
      '纯3km数值模式的24小时绝对误差中位数：',median_absolute_error(res['temperature'],res['Temperature2m']))
print('纯3km数值模式的48小时平均绝对误差：',mean_absolute_error(res['temperature'],res['Temperature2m_prefcst']),
      '纯3km数值模式的48小时绝对误差中位数：',median_absolute_error(res['temperature'],res['Temperature2m_prefcst']))

线性方案的平均绝对误差： 0.6794100452432723 线性方案的绝对误差中位数： 0.598464133791623
纯3km数值模式的24小时平均绝对误差： 0.9033707410752606 纯3km数值模式的24小时绝对误差中位数： 0.813294987754233
纯3km数值模式的48小时平均绝对误差： 1.2020320388074675 纯3km数值模式的48小时绝对误差中位数： 1.1226886192727363


In [ ]:
##########################
##########################
##########################

In [ ]:
#为了更详实的验证，添加了一块用现有模型不进行额外训练直接用5月份的模式预报结果进行预报验证。

In [7]:
path = 'I:\data_new_synop05'
filelist = os.listdir(path)
filelist.sort()

In [8]:
records05 = extract_obs_data(path,filelist)

one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record
one record

In [12]:
with open('records05.pkl','wb') as fp:
    pickle.dump(records05,fp)

In [19]:
df05 = pd.DataFrame(records05,columns=features).set_index('date')

In [22]:
fcstpath = os.path.abspath('I:\mars3km_00_48h/')

In [25]:
fcst_records05=[]
idx=0
for date in df05.index:
    validtime = date.to_pydatetime()
    fcst_date_before=date.date() - forcast_d
    fcst_date=dateparser.parse(date.date().strftime("%Y/%m/%d-%H:%M:%S"))
    fcst_date_before=dateparser.parse(fcst_date_before.strftime("%Y/%m/%d-%H:%M:%S"))
    
    timedelta = validtime - fcst_date
    timedelta_before = validtime - fcst_date_before
    
    timedelta=int(timedelta.total_seconds()/3600)
    timedelta_before=int(timedelta_before.total_seconds()/3600)
    
    filename1=fcstpath+os.sep+"%s%03d"%(fcst_date.strftime("%Y%m%d%H/mars3km%y%m%d%H"),timedelta)
    filename2=fcstpath+os.sep+"%s%03d"%(fcst_date_before.strftime("%Y%m%d%H/mars3km%y%m%d%H"),timedelta_before)
    if os.path.isfile(filename1) and os.path.isfile(filename2):
        f1=forcast_to_result(ctl,filename1,lonlat_in_guangdong)
        f2=forcast_to_result(ctl,filename2,lonlat_in_guangdong)
        fcst_records05.append(FcstSummary(
            date                   =validtime,
            CombinedRadarReflective=f1.cr,
            SurfacePressure        =f1.ps,
            SeaLevelPressure       =f1.psl,
            RelativeHumidity2m     =f1.rh2m,
            Temperature2m          =f1.t2m,
            SurfaceTemperature     =f1.ts,
            U_wind10m              =f1.u10m,
            V_wind10m              =f1.v10m,
            CombinedRadarReflective_prefcst=f2.cr,
            SurfacePressure_prefcst        =f2.ps,
            SeaLevelPressure_prefcst       =f2.psl,
            RelativeHumidity2m_prefcst     =f2.rh2m,
            Temperature2m_prefcst          =f2.t2m,
            SurfaceTemperature_prefcst     =f2.ts,
            U_wind10m_prefcst              =f2.u10m,
            V_wind10m_prefcst              =f2.v10m,
        ))
        idx+=1
        print('num:',idx)

num: 1
num: 2
num: 3
num: 4
num: 5
num: 6
num: 7
num: 8
num: 9
num: 10
num: 11
num: 12
num: 13
num: 14
num: 15
num: 16
num: 17
num: 18
num: 19
num: 20
num: 21
num: 22
num: 23
num: 24
num: 25
num: 26
num: 27
num: 28
num: 29
num: 30
num: 31
num: 32
num: 33
num: 34
num: 35
num: 36
num: 37
num: 38
num: 39
num: 40
num: 41
num: 42
num: 43
num: 44
num: 45
num: 46
num: 47
num: 48
num: 49
num: 50
num: 51
num: 52
num: 53
num: 54
num: 55
num: 56
num: 57
num: 58
num: 59
num: 60
num: 61
num: 62
num: 63
num: 64
num: 65
num: 66
num: 67
num: 68
num: 69
num: 70
num: 71
num: 72
num: 73
num: 74
num: 75
num: 76
num: 77
num: 78
num: 79
num: 80
num: 81
num: 82
num: 83
num: 84
num: 85
num: 86
num: 87
num: 88
num: 89
num: 90
num: 91
num: 92
num: 93
num: 94
num: 95
num: 96
num: 97
num: 98
num: 99
num: 100
num: 101
num: 102
num: 103
num: 104
num: 105
num: 106
num: 107
num: 108
num: 109
num: 110
num: 111
num: 112
num: 113
num: 114
num: 115
num: 116
num: 117
num: 118
num: 119
num: 120
num: 121
num: 122
num: 123
n

In [27]:
with open('fcst_records05.pkl','wb') as fp:
    pickle.dump(fcst_records05,fp)

In [38]:
with open('fcst_records05.pkl','rb') as fp:
    fcst_records05=pickle.load(fp)
    
with open('records05.pkl','rb') as fp:
    records05=pickle.load(fp)

In [57]:
df_fcst05 = pd.DataFrame(fcst_records05,columns=fcst_features).set_index('date')
for feature in fcst_features:
    if feature != 'date':
        for N in range(1,4):
            derive_ntime_feature(df_fcst05,feature,N)

df05 = pd.DataFrame(records05,columns=features).set_index('date')
df_all05 = pd.concat([df05['temperature'],df_fcst05[X_test.columns],df_fcst05['Temperature2m_prefcst']],axis=1).dropna()

In [62]:
prediction05=regressor.predict(df_all05[X_test.columns])
res05 = pd.concat([df_all05[['temperature','Temperature2m','Temperature2m_prefcst']],pd.DataFrame({'end_var':prediction05},index=df_all05.index)],axis=1)
res05 = res05 - 273.15

In [63]:
res05.plot()

In [61]:
from sklearn.metrics import mean_absolute_error,median_absolute_error
print('线性方案的平均绝对误差：',mean_absolute_error(res05['temperature'],res05['end_var']),
      '线性方案的绝对误差中位数：',median_absolute_error(res05['temperature'],res05['end_var']))
print('纯3km数值模式的24小时平均绝对误差：',mean_absolute_error(res05['temperature'],res05['Temperature2m']),
      '纯3km数值模式的24小时绝对误差中位数：',median_absolute_error(res05['temperature'],res05['Temperature2m']))
print('纯3km数值模式的48小时平均绝对误差：',mean_absolute_error(res05['temperature'],res05['Temperature2m_prefcst']),
      '纯3km数值模式的48小时绝对误差中位数：',median_absolute_error(res05['temperature'],res05['Temperature2m_prefcst']))

线性方案的平均绝对误差： 0.8157841609809573 线性方案的绝对误差中位数： 0.731798992141421
纯3km数值模式的24小时平均绝对误差： 1.0200630563686308 纯3km数值模式的24小时绝对误差中位数： 0.9635428133877895
纯3km数值模式的48小时平均绝对误差： 1.2639288400894908 纯3km数值模式的48小时绝对误差中位数： 1.224324625651036
